# Multi Agent Team Approach - With Termination Condition
We will be using 3 agents to now write the story, each with its own work and mastery

In [8]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage, MultiModalMessage
from autogen_core import Image as AGImage
from autogen_agentchat.teams import RoundRobinGroupChat, SelectorGroupChat
from autogen_agentchat.conditions import MaxMessageTermination,TextMentionTermination
from autogen_agentchat.base import TaskResult

from pydantic import BaseModel
from PIL import Image
from io import BytesIO
import requests

from IPython.display import Markdown, display
from dotenv import load_dotenv
import os

load_dotenv()
google_api_key = os.getenv('GOOGLE_API_KEY')
model_name = os.getenv("GEMINI_MODEL")

### Model Client

In [9]:
model_client = OpenAIChatCompletionClient(
    model=model_name,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
    api_key=google_api_key,
    model_info={
        "vision": True,
        "function_calling": True,
        "json_output": True,
        "family": "gpt-4o",
        "structured_output": True
    },
)

### Agents

In [10]:
plot_agent = AssistantAgent(
    name = 'plot_writer',
    model_client=model_client,
    system_message="You create engaging plots for stories. Focus on the Pokemon's journey"
)

character_agent = AssistantAgent(
    name = 'character_writer',
    model_client=model_client,
    system_message="You develop characters. Describe the pokemon and the villian in detail, including their motivations and backgrounds."
)

ending_agent = AssistantAgent(
    name = 'ending_writer',
    model_client=model_client,
    system_message="You wrute engaging endings. conclude the story with a twist."
)

review_agent = AssistantAgent(
    name = 'review_writer',
    model_client=model_client,
    system_message="You have to say 'APPROVE' once the story is complete and you like the full flow else provide your feedback."
)

In [11]:
termination_condition = MaxMessageTermination(10) | TextMentionTermination('APPROVE')


### Create a team

In [12]:
team = RoundRobinGroupChat(
    [plot_agent, character_agent, ending_agent,review_agent],
    termination_condition=termination_condition
    # max_turns=3
)

In [13]:
# When running inside a script, use a async main function and call it from `asyncio.run(...)`.
await team.reset()  # Reset the team for a new task.
async for message in team.run_stream(task="Write a short poem about the fall season."):  # type: ignore
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)

id='edfb0b65-a618-4221-b971-b224a41c9a0d' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 8, 6, 7, 41, 14, 349868, tzinfo=datetime.timezone.utc) content='Write a short poem about the fall season.' type='TextMessage'
id='445938e5-ff70-4304-9e60-9fd5952f30a2' source='plot_writer' models_usage=RequestUsage(prompt_tokens=25, completion_tokens=77) metadata={} created_at=datetime.datetime(2025, 8, 6, 7, 41, 16, 495661, tzinfo=datetime.timezone.utc) content="The leaves, they blush in crimson hue,\nA painter's stroke, both bold and true.\nThe sun, a gentler, golden gleam,\nWeaves through the branches, a waking dream.\n\nA crispness kisses morning air,\nA whispered promise, everywhere.\nThe world prepares for slumber deep,\nWhile autumn's beauty we will keep." type='TextMessage'
id='429f5f9a-fa91-426d-9284-1d6f740c245a' source='character_writer' models_usage=RequestUsage(prompt_tokens=109, completion_tokens=1035) metadata={} created_at=datetime.datetime(2025, 8, 6